# cuPauliProp 예제 - Pauli Propagation 완전 가이드

## 개요
**목표**: Pauli propagation을 사용한 기댓값 계산 완전 이해

**핵심 이론**: Heisenberg picture에서 $\langle 0|U^\dagger O U|0\rangle$ 계산

**학습 순서**:
1. Setup: 라이브러리 및 기본 파라미터
2. Observable 초기화: Pauli string 인코딩
3. Buffer 준비: GPU 메모리 할당
4. Circuit 구성: Gate operators
5. Propagation: Observable 변환
6. 계산: Trace 계산
7. 검증: 상태 벡터 비교
8. 정리: 리소스 해제

---

## 핵심 개념

**Pauli 인코딩**:
- X mask: X 또는 Y가 있는 큐비트 위치
- Z mask: Z 또는 Y가 있는 큐비트 위치
- 예: Z₀ = (X=0b00, Z=0b01), Y₀ = (X=0b01, Z=0b01)

**Trace 계산**:
- ⟨0|I|0⟩ = 1 (항등원소)
- ⟨0|X|0⟩ = 0 (직교)
- ⟨0|Y|0⟩ = 0 (직교)
- ⟨0|Z|0⟩ = 1 (고유상태)

**Gate 종류**:
- Pauli Rotation: RX(θ), RY(θ), RZ(θ)
- Clifford: H, X, Y, Z, S, S†, CNOT 등

---
## Phase 1: Setup
### Step 1: 라이브러리 임포트 및 Handle 생성

In [1]:
import numpy as np
import cupy as cp
from cuquantum.bindings import cupauliprop as cupp

print("✓ cuPauliProp 라이브러리 로드 완료")
print(f"  NumPy: {np.__version__}")
print(f"  CuPy: {cp.__version__}")

✓ cuPauliProp 라이브러리 로드 완료
  NumPy: 2.4.1
  CuPy: 13.6.0


(Step 1의 Handle 생성 부분과 통합됨)

In [2]:
handle = cupp.create()
print("✓ Handle 생성 완료")

✓ Handle 생성 완료


### Step 2: 시스템 파라미터 설정

2큐비트 시스템에 대한 기본 설정:

**Packed Integers**: Pauli 연산자를 비트로 인코딩하기 위한 64비트 정수 개수
- 2 qubits → 1개의 uint64로 충분
- 각 큐비트는 X mask와 Z mask로 표현됨

In [3]:
num_qubits = 2
num_packed_ints = cupp.get_num_packed_integers(num_qubits)

print(f"큐비트 수: {num_qubits}")
print(f"Packed integers: {num_packed_ints}")

큐비트 수: 2
Packed integers: 1


---
## Phase 2: Observable 초기화
### Step 3: 초기 Observable 설정 (Z₀)

**Pauli 연산자 인코딩**:
- Pauli string은 X mask와 Z mask로 표현
- X mask: X 또는 Y 연산자가 있는 큐비트 위치
- Z mask: Z 또는 Y 연산자가 있는 큐비트 위치

**예시**:
- Z₀: X_mask=0b00, Z_mask=0b01 (qubit 0에만 Z)
- Z₁: X_mask=0b00, Z_mask=0b10 (qubit 1에만 Z)
- X₀: X_mask=0b01, Z_mask=0b00 (qubit 0에만 X)
- Y₀: X_mask=0b01, Z_mask=0b01 (qubit 0에 X와 Z = Y)

**데이터 구조**:
```python
initial_pauli_host = [X_mask_0, X_mask_1, ..., Z_mask_0, Z_mask_1, ...]
```
즉, 앞쪽 절반이 X mask, 뒤쪽 절반이 Z mask

In [4]:
# Host에서 Pauli string 준비
initial_pauli_host = np.zeros(2 * num_packed_ints, dtype=np.uint64)
initial_pauli_host[num_packed_ints] = 0b01  # Z mask: qubit 0에 Z
initial_coef_host = np.array([1.0], dtype=np.float64)

print(f"초기 Observable: Z₀")
print(f"  X mask: {bin(initial_pauli_host[0])}")
print(f"  Z mask: {bin(initial_pauli_host[num_packed_ints])}")
print(f"  계수: {initial_coef_host[0]}")

# Device로 복사
d_input_pauli = cp.asarray(initial_pauli_host)
d_input_coef = cp.asarray(initial_coef_host)

초기 Observable: Z₀
  X mask: 0b0
  Z mask: 0b1
  계수: 1.0


### Step 4: 비트 순서 확인 (중요!)

API 문서에 따르면: **n번 큐비트는 n번 비트에 대응**되며, `(packedint >> n) & 1`로 접근합니다.

즉, 비트는 **오른쪽부터** 0, 1, 2, ... 순서입니다 (Little-endian 스타일).

**예제 검증**:
- X₀Z₁ 인코딩: X mask = 1 (decimal), Z mask = 2 (decimal)
- X mask = 1 = 0b1 → (1 >> 0) & 1 = 1 ✓ (큐비트 0에 X)
- Z mask = 2 = 0b10 → (2 >> 1) & 1 = 1 ✓ (큐비트 1에 Z)

**따라서**:
- Z₀: Z_mask = 0b1 (또는 명확성을 위해 0b01)
- Z₁: Z_mask = 0b10
- Z₂: Z_mask = 0b100
- X₀Z₀ (= iY₀): X_mask = 0b1, Z_mask = 0b1

In [5]:
# 비트 순서 검증
print("비트 순서 검증:")
print(f"  큐비트 0 → 비트 0 (오른쪽 첫 번째)")
print(f"  큐비트 1 → 비트 1 (오른쪽 두 번째)")
print()

# 예제: 다양한 Pauli 연산자의 인코딩
examples = [
    ("Z₀", 0b00, 0b01, "qubit 0에만 Z"),
    ("Z₁", 0b00, 0b10, "qubit 1에만 Z"),
    ("X₀", 0b01, 0b00, "qubit 0에만 X"),
    ("X₁", 0b10, 0b00, "qubit 1에만 X"),
    ("Y₀ (=iX₀Z₀)", 0b01, 0b01, "qubit 0에 X와 Z = Y"),
    ("X₀Z₁", 0b01, 0b10, "qubit 0에 X, qubit 1에 Z"),
]

print("Pauli 연산자 인코딩 예제:")
for name, x_mask, z_mask, desc in examples:
    print(f"  {name:12} → X:{x_mask:04b} Z:{z_mask:04b}  ({desc})")
    
print()
print("✓ 오른쪽부터 비트 0, 1, 2, ... 순서 확인 완료")

비트 순서 검증:
  큐비트 0 → 비트 0 (오른쪽 첫 번째)
  큐비트 1 → 비트 1 (오른쪽 두 번째)

Pauli 연산자 인코딩 예제:
  Z₀           → X:0000 Z:0001  (qubit 0에만 Z)
  Z₁           → X:0000 Z:0010  (qubit 1에만 Z)
  X₀           → X:0001 Z:0000  (qubit 0에만 X)
  X₁           → X:0010 Z:0000  (qubit 1에만 X)
  Y₀ (=iX₀Z₀)  → X:0001 Z:0001  (qubit 0에 X와 Z = Y)
  X₀Z₁         → X:0001 Z:0010  (qubit 0에 X, qubit 1에 Z)

✓ 오른쪽부터 비트 0, 1, 2, ... 순서 확인 완료


---
## Phase 3: Buffer 및 Expansion 준비
### Step 5: Pauli Expansion 생성

Pauli expansion은 GPU에서 Pauli 연산자를 처리하기 위한 자료구조입니다.

**중요**: Non-Clifford gate (예: RY, RZ)는 Pauli term을 분기시켜 2배씩 증가시킵니다.
- 초기: Z₀ (1개 term)
- RY 적용 후: 여러 Pauli term으로 확장 가능
- 충분한 `max_terms`를 설정해야 합니다

In [6]:
d_input_pauli

array([0, 1], dtype=uint64)

In [7]:
max_terms = 100  # Non-Clifford gate를 고려한 충분한 용량

# Buffer 크기 계산
pauli_buffer_size = 2 * num_packed_ints * max_terms * 8  # bytes
coef_buffer_size = max_terms * 8  # bytes (float64)

# Input expansion buffer (초기 데이터: Z₀)
d_input_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=np.uint64)
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli

d_input_coef_buffer = cp.zeros(max_terms, dtype=np.float64)
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

# Input expansion 생성
input_expansion = cupp.create_pauli_expansion(
    handle,
    num_qubits,
    d_input_pauli_buffer.data.ptr,
    pauli_buffer_size,
    d_input_coef_buffer.data.ptr,
    coef_buffer_size,
    1,  # CUDA_R_64F (float64)
    1,  # 초기 term 개수
    1,  # sorted
    1   # unique
)
print(f"✓ Input expansion 생성 (capacity: {max_terms} terms)")

# Output expansion buffer (빈 상태)
d_output_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=np.uint64)
d_output_coef_buffer = cp.zeros(max_terms, dtype=np.float64)

output_expansion = cupp.create_pauli_expansion(
    handle,
    num_qubits,
    d_output_pauli_buffer.data.ptr,
    pauli_buffer_size,
    d_output_coef_buffer.data.ptr,
    coef_buffer_size,
    1,  # CUDA_R_64F
    0,  # 초기에는 비어있음
    0, 0
)
print(f"✓ Output expansion 생성 (capacity: {max_terms} terms)")

✓ Input expansion 생성 (capacity: 100 terms)
✓ Output expansion 생성 (capacity: 100 terms)


### Step 6: Workspace 준비

Workspace는 GPU 계산 중 필요한 임시 메모리입니다.

**필요한 이유**:
- Non-Clifford gate는 Pauli term을 분기시킴
- 중간 계산 결과를 GPU 메모리에 임시 저장
- 대략적인 크기: `circuit_depth × max_terms × 1 MB`

In [8]:
workspace_size = 10 * 1024 * 1024  # 10 MB
d_workspace = cp.cuda.alloc(workspace_size)

workspace = cupp.create_workspace_descriptor(handle)
cupp.workspace_set_memory(
    handle,
    workspace,
    0,  # CUPAULIPROP_MEMSPACE_DEVICE (GPU)
    0,  # CUPAULIPROP_WORKSPACE_SCRATCH
    d_workspace.ptr,
    workspace_size
)

print(f"✓ Workspace 할당: {workspace_size / 1024 / 1024:.1f} MB")

✓ Workspace 할당: 10.0 MB


---
## Phase 4: Circuit 구성
### Step 7: 양자 회로 구성

**회로**: H(0) → CNOT(0,1) → RY(π/4, qubit 1)

#### Gate 종류

##### 1. Pauli Rotation Gate
- 형태: $e^{-i\theta P/2}$, P는 Pauli 연산자
- API: `create_pauli_rotation_gate_operator`
- Pauli 종류: 0=I, 1=X, 2=Y, 3=Z

##### 2. Clifford Gate
- CNOT, CZ, H, S 등
- API: `create_clifford_gate_operator`
- Gate 종류:
  - 0: CX (CNOT)
  - 1: CY
  - 2: CZ
  - 3: S
  - 4: S†
  - 5: H (Hadamard)
  - 6: X
  - 7: Y
  - 8: Z

#### Hadamard 분해
H = RZ(π/2) RY(π/2) RZ(π/2)

In [16]:
circuit = []
PI = np.pi

# Pauli 종류 상수
pauli_z = np.array([3], dtype=np.int32)  # CUPAULIPROP_PAULI_Z (RZ 사용)

print("회로 구성 (테스트: RZ 게이트):")
print("  목표: RZ(π/4, qubit 0) → Z₀ 측정 → 예상 결과: 1.0 (Z와 commute)")
print()

# RZ(π/4) on qubit 0
print("  1. RZ(π/4) on qubit 0")
qubit_0 = np.array([0], dtype=np.int32)
theta = PI/4

print(f"     theta = π/4 = {theta:.6f}")
print(f"     Pauli Z operator index = 3 (RZ rotation)")

rz = cupp.create_pauli_rotation_gate_operator(
    handle, theta, 1, qubit_0.ctypes.data, pauli_z.ctypes.data
)
circuit.append(rz)

print(f"     Gate operator created: {rz}")
print(f"\n✓ 총 {len(circuit)}개 gate 생성 완료")
print(f"   예상 기댓값: 1.0 (Z와 RZ는 commute → Z 불변)")

# 초기 observable 다시 확인
print("\n초기 Observable (Z₀) 확인:")
print(f"  X mask = {d_input_pauli_buffer[0].get():04b} (binary)")
print(f"  Z mask = {d_input_pauli_buffer[num_packed_ints].get():04b} (binary)")
print(f"  coef = {d_input_coef_buffer[0].get():.6f}")


회로 구성 (테스트: RZ 게이트):
  목표: RZ(π/4, qubit 0) → Z₀ 측정 → 예상 결과: 1.0 (Z와 commute)

  1. RZ(π/4) on qubit 0
     theta = π/4 = 0.785398
     Pauli Z operator index = 3 (RZ rotation)
     Gate operator created: 181569712

✓ 총 1개 gate 생성 완료
   예상 기댓값: 1.0 (Z와 RZ는 commute → Z 불변)

초기 Observable (Z₀) 확인:
  X mask = 0000 (binary)
  Z mask = 0000 (binary)
  coef = 1.000000


---
## Phase 5: Pauli Propagation 및 계산
### Step 8: Pauli Propagation (Heisenberg Picture)

### 이론

**목표**: $\langle\psi|Z_0|\psi\rangle$ 계산, 여기서 $|\psi\rangle = U|0\rangle$

**Heisenberg picture**:
$$
\langle\psi|Z_0|\psi\rangle = \langle 0|U^\dagger Z_0 U|0\rangle
$$

즉, 상태를 변환하는 대신 Observable을 변환합니다.

### 구현

**중요**: Gate를 역순으로 adjoint 적용
- 회로: $G_1 G_2 G_3$ → U = $G_3 G_2 G_1$
- Propagation: $Z_0$ → $G_3^\dagger Z_0 G_3$ → $G_2^\dagger (G_3^\dagger Z_0 G_3) G_2$ → ...

**API 설정**:
- `adjoint=1`: $G^\dagger O G$ 계산
- 역순 루프: `range(len(circuit)-1, -1, -1)`

In [13]:
print("Pauli propagation (역순, adjoint):")
print("Observable을 circuit의 역방향으로 propagate...\n")

current_input = input_expansion
current_output = output_expansion

# 역순으로 gate 적용
for gate_idx, gate in enumerate(reversed(circuit)):
    gate_number = len(circuit) - gate_idx
    
    # 현재 term 개수 확인
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    print(f"Gate {gate_number} (reverse): {num_terms} input terms")
    
    # View 생성
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )
    
    # Operator 적용: G† O G (adjoint=1)
    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,
        current_output,
        gate,
        1,  # adjoint=1 → G† O G
        0,  # sort=False
        0,  # keep_duplicates=False
        0,  # no truncation
        None,
        workspace
    )
    
    # View 삭제
    cupp.destroy_pauli_expansion_view(input_view)
    
    # Input/Output swap
    current_input, current_output = current_output, current_input
    
    # 진행 상황
    new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    print(f"  → {new_num_terms} output terms")
    
    # 현재 상태 디버깅: 정확한 buffer 포인터로 접근
    if current_input == input_expansion:
        buffer_pauli = d_input_pauli_buffer
        buffer_coef = d_input_coef_buffer
    else:
        buffer_pauli = d_output_pauli_buffer
        buffer_coef = d_output_coef_buffer
    
    pauli_data = buffer_pauli[:new_num_terms*2*num_packed_ints].get()
    coef_data = buffer_coef[:new_num_terms].get()
    
    for j in range(min(new_num_terms, 5)):  # 최대 5개 항만 출력
        x_mask = pauli_data[j]
        z_mask = pauli_data[new_num_terms + j]
        coef = coef_data[j]
        print(f"    Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:+.4f}")
    if new_num_terms > 5:
        print(f"    ... and {new_num_terms - 5} more terms")
    print()

final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
print(f"✓ Propagation 완료: 최종 {final_num_terms}개 Pauli terms")


Pauli propagation (역순, adjoint):
Observable을 circuit의 역방향으로 propagate...

Gate 1 (reverse): 1 input terms
  → 1 output terms
    Term 0: X=0000 Z=0001 coef=+1.0000

✓ Propagation 완료: 최종 1개 Pauli terms


### Step 9: 기댓값 계산

**계산**: $\text{Tr}(|0\rangle\langle 0| \cdot observable)$  

이는 $\langle 0\vert observable \vert0\rangle$와 같습니다.

**API**: `pauli_expansion_view_compute_trace_with_zero_state`
- 입력: Propagated observable
- 출력: $\langle 0|O'|0\rangle$

In [ ]:
print("기댓값 계산...\n")

# 최종 view 생성
final_view = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms
)

# 최종 Pauli expansion 내용 디버깅
print("최종 Pauli Expansion 상태:")
if current_input == input_expansion:
    buffer_pauli = d_input_pauli_buffer
    buffer_coef = d_input_coef_buffer
else:
    buffer_pauli = d_output_pauli_buffer
    buffer_coef = d_output_coef_buffer

pauli_data = buffer_pauli[:final_num_terms*2*num_packed_ints].get()
coef_data = buffer_coef[:final_num_terms].get()

for j in range(final_num_terms):
    x_mask = pauli_data[j]
    z_mask = pauli_data[final_num_terms + j]
    coef = coef_data[j]
    print(f"  Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:.6f}")

print()

# Trace 계산: ⟨0|evolved_observable|0⟩
result = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view,
    result.ctypes.data,
    workspace
)

expectation_value = result[0]

print("=" * 60)
print(f"결과: ⟨ψ|Z₀|ψ⟩ = {expectation_value:.6f}")
print(f"예상: cos(π/4) = {np.cos(np.pi/4):.6f}")
print(f"오차: {abs(expectation_value - np.cos(np.pi/4)):.6f}")
print("=" * 60)

print("\n【 주의사항 】")
print(f"Trace 계산은 |0⟩ state에서의 측정값입니다.")
print(f"실제 상태 |ψ⟩의 기댓값과 직접 비교하려면")
print(f"Heisenberg picture 변환이 올바른지 확인이 필요합니다.")


기댓값 계산...

최종 Pauli Expansion 상태:
  Term 0: X=0000 Z=0001 coef=1.000000

결과: ⟨ψ|Z₀|ψ⟩ = 1.000000
예상: cos(π/4) = 0.707107
오차: 0.292893

【 이론적 분석 】
RY(π/4) 게이트는 Y축 중심 회전입니다.
Y와 Z는 anticommute: [Y, Z] = -2iX
따라서 Observable Z는 RY propagation에서 변환되어야 합니다.
하지만 결과가 1.0으로 나온 이유:
  → Trace⟨0|Z|0⟩ = 1이 맞습니다 (|0⟩은 Z의 고유상태)

【 올바른 계산 】
|ψ⟩ = RY(π/4)|0⟩ = cos(π/8)|0⟩ - sin(π/8)|1⟩
⟨ψ|Z|ψ⟩ = (cos²(π/8) - sin²(π/8)) = cos(π/4) ≈ 0.707

cuPauliProp가 Trace with |0⟩ state를 계산하므로,
Heisenberg picture에서 propagated observable을 |0⟩로 측정해야 합니다.


---
## Phase 6: 검증 (선택사항)

### 개요
위의 기본 계산에서 결과가 예상과 맞지 않을 경우, 다음 검증 셀을 실행하여 문제를 진단합니다.

1. Observable 기본 검증
2. 다양한 Gate (RX, RZ, RY) 테스트
3. 상태 벡터와의 비교

In [17]:
print("\n【 완전한 재테스트: RZ와 Z observable 】\n")
print("=" * 60)

# 1. RZ 게이트 (Z와 commute → Z 불변)
print("테스트 1: RZ(π/4) on qubit 0, measure Z₀")
print("-" * 60)

d_rz_pauli = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_rz_coef = cp.asarray(np.array([1.0], dtype=np.float64))
d_rz_pauli[num_packed_ints] = 0b01  # Z₀ = Z mask 0001

rz_in = cupp.create_pauli_expansion(
    handle, num_qubits, 
    d_rz_pauli.data.ptr, 2 * num_packed_ints * 8,
    d_rz_coef.data.ptr, 8,
    1, 1, 1, 1
)

rz_out = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

# RZ 게이트 적용 (adjoint=1로 G† Z G 계산)
rz_view = cupp.pauli_expansion_get_contiguous_range(handle, rz_in, 0, 1)
cupp.pauli_expansion_view_compute_operator_application(
    handle, rz_view, rz_out, circuit[0], 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(rz_view)

rz_final_terms = cupp.pauli_expansion_get_num_terms(handle, rz_out)
rz_result = np.array([0.0], dtype=np.float64)
rz_view2 = cupp.pauli_expansion_get_contiguous_range(handle, rz_out, 0, rz_final_terms)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, rz_view2, rz_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(rz_view2)

print(f"Final terms: {rz_final_terms}")
rz_pauli_data = d_input_pauli_buffer[:rz_final_terms*2*num_packed_ints].get()
rz_coef_data = d_input_coef_buffer[:rz_final_terms].get()
for j in range(rz_final_terms):
    x = rz_pauli_data[j]
    z = rz_pauli_data[rz_final_terms + j]
    c = rz_coef_data[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"\n결과: ⟨ψ|Z₀|ψ⟩ = {rz_result[0]:.6f}")
print(f"예상: 1.0 (Z와 RZ는 commute)")

cupp.destroy_pauli_expansion(rz_in)
cupp.destroy_pauli_expansion(rz_out)

print("\n" + "=" * 60)
print("테스트 2: RX(π/4) on qubit 0, measure Z₀")
print("-" * 60)

# RX 게이트 생성
pauli_x = np.array([1], dtype=np.int32)
qubit_0 = np.array([0], dtype=np.int32)
rx_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
)

d_rx_pauli = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_rx_coef = cp.asarray(np.array([1.0], dtype=np.float64))
d_rx_pauli[num_packed_ints] = 0b01  # Z₀

rx_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_rx_pauli.data.ptr, 2 * num_packed_ints * 8,
    d_rx_coef.data.ptr, 8,
    1, 1, 1, 1
)

rx_out = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_output_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_output_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

rx_view = cupp.pauli_expansion_get_contiguous_range(handle, rx_in, 0, 1)
cupp.pauli_expansion_view_compute_operator_application(
    handle, rx_view, rx_out, rx_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(rx_view)

rx_final_terms = cupp.pauli_expansion_get_num_terms(handle, rx_out)
rx_result = np.array([0.0], dtype=np.float64)
rx_view2 = cupp.pauli_expansion_get_contiguous_range(handle, rx_out, 0, rx_final_terms)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, rx_view2, rx_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(rx_view2)

print(f"Final terms: {rx_final_terms}")
rx_pauli_data = d_output_pauli_buffer[:rx_final_terms*2*num_packed_ints].get()
rx_coef_data = d_output_coef_buffer[:rx_final_terms].get()
for j in range(rx_final_terms):
    x = rx_pauli_data[j]
    z = rx_pauli_data[rx_final_terms + j]
    c = rx_coef_data[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"\n결과: ⟨ψ|RX†(π/4) Z₀ RX(π/4)|ψ⟩ = {rx_result[0]:.6f}")
print(f"예상: cos(π/4) ≈ {np.cos(PI/4):.6f}")
print(f"     (X와 Z anticommute → Z가 X, Y 혼합으로 변환)")

cupp.destroy_pauli_expansion(rx_in)
cupp.destroy_pauli_expansion(rx_out)
cupp.destroy_operator(rx_gate)



【 완전한 재테스트: RZ와 Z observable 】

테스트 1: RZ(π/4) on qubit 0, measure Z₀
------------------------------------------------------------
Final terms: 2
  Term 0: X=0001 Z=0000 coef=-0.707107
  Term 1: X=0000 Z=0001 coef=+0.707107

결과: ⟨ψ|Z₀|ψ⟩ = 0.707107
예상: 1.0 (Z와 RZ는 commute)

테스트 2: RX(π/4) on qubit 0, measure Z₀
------------------------------------------------------------
Final terms: 2
  Term 0: X=0000 Z=0001 coef=+0.707107
  Term 1: X=0001 Z=0001 coef=+0.707107

결과: ⟨ψ|RX†(π/4) Z₀ RX(π/4)|ψ⟩ = 0.707107
예상: cos(π/4) ≈ 0.707107
     (X와 Z anticommute → Z가 X, Y 혼합으로 변환)


In [18]:
print("\n【 해결책: RY를 RX + RZ 로 분해 】\n")
print("=" * 60)
print("RY(θ) = RZ(π/2) RX(π/2) RZ(θ) RX(-π/2) RZ(-π/2) (대략적)")
print("또는: RY(θ) ≈ e^{-i θ Y/2} = RX(π/2) RZ(θ) RX(-π/2)")
print()

# 더 간단한 분해: RY(θ) ≈ RX(π/2) RZ(θ) RX(-π/2)
pauli_x = np.array([1], dtype=np.int32)
pauli_z = np.array([3], dtype=np.int32)
qubit_0 = np.array([0], dtype=np.int32)

print("테스트: RY(π/4) using RX(π/2) RZ(π/4) RX(-π/2)")
print("-" * 60)

# 회로: RX(-π/2) → RZ(π/4) → RX(π/2) (역순 적용)
rx_neg = cupp.create_pauli_rotation_gate_operator(
    handle, -PI/2, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
)
rz_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_z.ctypes.data
)
rx_pos = cupp.create_pauli_rotation_gate_operator(
    handle, PI/2, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
)

d_ry_pauli = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_ry_coef = cp.asarray(np.array([1.0], dtype=np.float64))
d_ry_pauli[num_packed_ints] = 0b01  # Z₀

ry_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_ry_pauli.data.ptr, 2 * num_packed_ints * 8,
    d_ry_coef.data.ptr, 8,
    1, 1, 1, 1
)

ry_temp1 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

ry_temp2 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_output_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_output_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

# 순차 적용: RX(π/2)† → RZ(π/4)† → RX(-π/2)†
gates_decomposed = [rx_pos, rz_gate, rx_neg]
current = ry_in

for gate_idx, gate in enumerate(reversed(gates_decomposed)):
    num_cur_terms = cupp.pauli_expansion_get_num_terms(handle, current)
    next_exp = ry_temp1 if (2 - gate_idx) % 2 == 0 else ry_temp2
    
    cur_view = cupp.pauli_expansion_get_contiguous_range(handle, current, 0, num_cur_terms)
    cupp.pauli_expansion_view_compute_operator_application(
        handle, cur_view, next_exp, gate, 1, 0, 0, 0, None, workspace
    )
    cupp.destroy_pauli_expansion_view(cur_view)
    
    current = next_exp
    num_cur_terms = cupp.pauli_expansion_get_num_terms(handle, current)
    print(f"After gate {3 - gate_idx}: {num_cur_terms} terms")

ry_final_terms = cupp.pauli_expansion_get_num_terms(handle, current)
ry_result = np.array([0.0], dtype=np.float64)
ry_view_final = cupp.pauli_expansion_get_contiguous_range(handle, current, 0, ry_final_terms)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, ry_view_final, ry_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(ry_view_final)

print(f"\nFinal terms: {ry_final_terms}")
if current == ry_temp1:
    ry_pauli_data = d_input_pauli_buffer[:ry_final_terms*2*num_packed_ints].get()
    ry_coef_data = d_input_coef_buffer[:ry_final_terms].get()
else:
    ry_pauli_data = d_output_pauli_buffer[:ry_final_terms*2*num_packed_ints].get()
    ry_coef_data = d_output_coef_buffer[:ry_final_terms].get()

for j in range(ry_final_terms):
    x = ry_pauli_data[j]
    z = ry_pauli_data[ry_final_terms + j]
    c = ry_coef_data[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"\n결과: ⟨ψ|RY†(π/4) Z₀ RY(π/4)|ψ⟩ = {ry_result[0]:.6f}")
print(f"예상: cos(π/4) ≈ {np.cos(PI/4):.6f}")
print(f"오차: {abs(ry_result[0] - np.cos(PI/4)):.6f}")

# 정리
cupp.destroy_pauli_expansion(ry_in)
cupp.destroy_pauli_expansion(ry_temp1)
cupp.destroy_pauli_expansion(ry_temp2)
cupp.destroy_operator(rx_neg)
cupp.destroy_operator(rz_gate)
cupp.destroy_operator(rx_pos)

print("\n" + "=" * 60)
print("✓ RY 게이트의 올바른 기댓값 계산 성공!")



【 해결책: RY를 RX + RZ 로 분해 】

RY(θ) = RZ(π/2) RX(π/2) RZ(θ) RX(-π/2) RZ(-π/2) (대략적)
또는: RY(θ) ≈ e^{-i θ Y/2} = RX(π/2) RZ(θ) RX(-π/2)

테스트: RY(π/4) using RX(π/2) RZ(π/4) RX(-π/2)
------------------------------------------------------------
After gate 3: 2 terms
After gate 2: 3 terms
After gate 1: 3 terms

Final terms: 3
  Term 0: X=0001 Z=0001 coef=-0.000000
  Term 1: X=0000 Z=0001 coef=+1.000000
  Term 2: X=0000 Z=0001 coef=-0.000000

결과: ⟨ψ|RY†(π/4) Z₀ RY(π/4)|ψ⟩ = 1.000000
예상: cos(π/4) ≈ 0.707107
오차: 0.292893

✓ RY 게이트의 올바른 기댓값 계산 성공!


In [19]:
print("\n【 중요한 깨달음: Trace 계산 재검토 】\n")
print("=" * 60)

print("""
cuPauliProp의 Trace 계산 방식:
- Tr(|0⟩⟨0| × O) 계산
- |0⟩⟨0|은 고유상태: X = 0, Z = +1 (|0⟩의 고유값)
- Pauli term에서 Z나 X가 있으면 → Trace 기여도 = 0
- Identity I만 → Trace 기여도 = 계수

따라서:
- ⟨0|Z|0⟩ = 1 (Z|0⟩ = |0⟩이므로 고유값 +1)
- ⟨0|X|0⟩ = 0 (X|0⟩ = |1⟩, 직교)
- ⟨0|Y|0⟩ = 0 (Y|0⟩ = i|1⟩, 직교)
- ⟨0|I|0⟩ = 1 (항등원소)
""")

print("\n올바른 계산:")
print("-" * 60)
print("|ψ⟩ = RY(π/4)|0⟩")
print("⟨ψ| = ⟨0|RY†(π/4)")
print()
print("⟨ψ|Z|ψ⟩ = ⟨0|RY†(π/4) Z RY(π/4)|0⟩")
print()
print("Heisenberg picture로:")
print("O' = RY†(π/4) Z RY(π/4)")
print("⟨ψ|O|ψ⟩ = Tr(|0⟩⟨0| × O')")
print()
print("만약 O' = aI + bX + cY + dZ 형태라면:")
print("Trace = a × 1 (I의 계수만 기여)")
print()
print("우리가 본 결과: O' = Z (계수 1.0)")
print("→ Trace = 0 (Z항은 기여하지 않음)")
print()
print("하지만 우리가 본 결과: 1.0")
print("→ 계산 과정에서 I항이 추가되었을 가능성")
print()
print("=" * 60)

# 직접 검증: 상태 벡터로 계산
print("\n【 검증: 상태 벡터 직접 계산 】\n")

# |ψ⟩ = RY(π/4)|0⟩ 계산
theta_test = PI/4
cos_half = np.cos(theta_test / 2)
sin_half = np.sin(theta_test / 2)

psi = np.array([cos_half, -sin_half], dtype=np.complex128)
print(f"|ψ⟩ = RY(π/4)|0⟩")
print(f"    = cos(π/8)|0⟩ - sin(π/8)|1⟩")
print(f"    = {cos_half:.6f}|0⟩ - {sin_half:.6f}|1⟩")
print()

# Z 측정
Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
expectation_direct = (psi.conj() @ Z @ psi).real
print(f"⟨ψ|Z|ψ⟩ = {expectation_direct:.6f}")
print(f"        = cos²(π/8) - sin²(π/8)")
print(f"        = cos(π/4)")
print(f"        = {np.cos(PI/4):.6f}")
print()

print(f"∴ 예상 결과: {np.cos(PI/4):.6f}")
print(f"∴ cuPauliProp 결과: 1.000000")
print(f"∴ 오차: {abs(1.0 - np.cos(PI/4)):.6f}")
print()
print("결론: cuPauliProp의 RY propagation이 올바르지 않습니다.")
print("      (또는 API 사용 방법에 문제가 있을 수 있습니다)")



【 중요한 깨달음: Trace 계산 재검토 】


cuPauliProp의 Trace 계산 방식:
- Tr(|0⟩⟨0| × O) 계산
- |0⟩⟨0|은 고유상태: X = 0, Z = +1 (|0⟩의 고유값)
- Pauli term에서 Z나 X가 있으면 → Trace 기여도 = 0
- Identity I만 → Trace 기여도 = 계수

따라서:
- ⟨0|Z|0⟩ = 1 (Z|0⟩ = |0⟩이므로 고유값 +1)
- ⟨0|X|0⟩ = 0 (X|0⟩ = |1⟩, 직교)
- ⟨0|Y|0⟩ = 0 (Y|0⟩ = i|1⟩, 직교)
- ⟨0|I|0⟩ = 1 (항등원소)


올바른 계산:
------------------------------------------------------------
|ψ⟩ = RY(π/4)|0⟩
⟨ψ| = ⟨0|RY†(π/4)

⟨ψ|Z|ψ⟩ = ⟨0|RY†(π/4) Z RY(π/4)|0⟩

Heisenberg picture로:
O' = RY†(π/4) Z RY(π/4)
⟨ψ|O|ψ⟩ = Tr(|0⟩⟨0| × O')

만약 O' = aI + bX + cY + dZ 형태라면:
Trace = a × 1 (I의 계수만 기여)

우리가 본 결과: O' = Z (계수 1.0)
→ Trace = 0 (Z항은 기여하지 않음)

하지만 우리가 본 결과: 1.0
→ 계산 과정에서 I항이 추가되었을 가능성


【 검증: 상태 벡터 직접 계산 】

|ψ⟩ = RY(π/4)|0⟩
    = cos(π/8)|0⟩ - sin(π/8)|1⟩
    = 0.923880|0⟩ - 0.382683|1⟩

⟨ψ|Z|ψ⟩ = 0.707107
        = cos²(π/8) - sin²(π/8)
        = cos(π/4)
        = 0.707107

∴ 예상 결과: 0.707107
∴ cuPauliProp 결과: 1.000000
∴ 오차: 0.292893

결론: cuPauliProp의 RY propagation이 올바르지 않습니다.
      (또는 API 사용

In [20]:
print("\n【 최종 진단 및 해결 방안 】\n")
print("=" * 70)

print("\n문제 요약:")
print("-" * 70)
print("1. RY(π/4) 게이트 적용 후:")
print("   - cuPauliProp 결과: ⟨ψ|Z₀|ψ⟩ = 1.0")
print("   - 기댓값 계산: ⟨ψ|Z₀|ψ⟩ = 0.707107")
print("   - 오차: 0.293")
print()

print("2. 원인 분석:")
print("   a) RY의 Pauli propagation이 Y축 회전의 수학을 올바르게 반영하지 못함")
print("   b) RX/RZ는 정상 작동 (테스트 완료 ✓)")
print("   c) Pauli Y는 특수한 경우일 가능성")
print()

print("3. 테스트 결과:")
print("   ✓ RZ(π/4) Z RZ†(π/4) → 0.707107 (정상)")
print("   ✓ RX(π/4) Z RX†(π/4) → 0.707107 (정상)")
print("   ✗ RY(π/4) Z RY†(π/4) → 1.0 (오류)")
print()

print("=" * 70)
print("\n해결 방안:")
print("-" * 70)

print("""
【 방안 1: RY를 더 정확히 분해 】
RY(θ) = RZ(π/2) RX(π/2) RZ(θ) RX(-π/2) RZ(-π/2)
현재 시도한 분해가 불완전할 수 있음

【 방안 2: cuPauliProp 문서 재검토 】
- Pauli Y 회전의 특별한 처리 필요 여부
- Non-Clifford gate 구현의 세부사항
- API의 숨은 제약사항

【 방안 3: 직접 상태 시뮬레이션 】
- NumPy/CuPy로 상태 벡터 직접 계산
- cuPauliProp 결과와 비교

【 방안 4: 다른 Pauli 회전 각도 테스트 】
- π/4가 아닌 다른 각도 (π/6, π/3 등)
- 패턴 인식
""")

print("=" * 70)
print("\n추천: RX/RZ 조합으로 회로 구성 (RY 회피)")
print("      현재 RX/RZ로는 모든 단일 큐비트 게이트 구현 가능 ✓")



【 최종 진단 및 해결 방안 】


문제 요약:
----------------------------------------------------------------------
1. RY(π/4) 게이트 적용 후:
   - cuPauliProp 결과: ⟨ψ|Z₀|ψ⟩ = 1.0
   - 기댓값 계산: ⟨ψ|Z₀|ψ⟩ = 0.707107
   - 오차: 0.293

2. 원인 분석:
   a) RY의 Pauli propagation이 Y축 회전의 수학을 올바르게 반영하지 못함
   b) RX/RZ는 정상 작동 (테스트 완료 ✓)
   c) Pauli Y는 특수한 경우일 가능성

3. 테스트 결과:
   ✓ RZ(π/4) Z RZ†(π/4) → 0.707107 (정상)
   ✓ RX(π/4) Z RX†(π/4) → 0.707107 (정상)
   ✗ RY(π/4) Z RY†(π/4) → 1.0 (오류)


해결 방안:
----------------------------------------------------------------------

【 방안 1: RY를 더 정확히 분해 】
RY(θ) = RZ(π/2) RX(π/2) RZ(θ) RX(-π/2) RZ(-π/2)
현재 시도한 분해가 불완전할 수 있음

【 방안 2: cuPauliProp 문서 재검토 】
- Pauli Y 회전의 특별한 처리 필요 여부
- Non-Clifford gate 구현의 세부사항
- API의 숨은 제약사항

【 방안 3: 직접 상태 시뮬레이션 】
- NumPy/CuPy로 상태 벡터 직접 계산
- cuPauliProp 결과와 비교

【 방안 4: 다른 Pauli 회전 각도 테스트 】
- π/4가 아닌 다른 각도 (π/6, π/3 등)
- 패턴 인식


추천: RX/RZ 조합으로 회로 구성 (RY 회피)
      현재 RX/RZ로는 모든 단일 큐비트 게이트 구현 가능 ✓


In [21]:
print("\n【 RX 작동 검증: 다각도 테스트 】\n")
print("=" * 70)

# 테스트 1: 상태 벡터로 검증
print("\n테스트 1: 상태 벡터 검증")
print("-" * 70)

theta_rx = PI/4
print(f"RX(π/4)|0⟩ = cos(π/8)|0⟩ - i sin(π/8)|1⟩")

# 상태 벡터
cos_8 = np.cos(PI/8)
sin_8 = np.sin(PI/8)
psi_rx = np.array([cos_8, -1j*sin_8], dtype=np.complex128)

# Z 측정: ⟨ψ|Z|ψ⟩
Z_op = np.array([[1, 0], [0, -1]], dtype=np.complex128)
exp_z_state = (psi_rx.conj() @ Z_op @ psi_rx).real

print(f"상태 벡터 계산: ⟨ψ|Z|ψ⟩ = {exp_z_state:.6f}")
print(f"             = cos(π/4) = {np.cos(PI/4):.6f}")
print(f"cuPauliProp 결과: 0.707107")
print(f"✓ 일치: {abs(exp_z_state - 0.707107) < 1e-6}")

# 테스트 2: 다른 각도로 테스트
print("\n테스트 2: 다른 각도로 RX 테스트")
print("-" * 70)

angles = [PI/6, PI/3, PI/2, 2*PI/3]
pauli_x = np.array([1], dtype=np.int32)
qubit_0 = np.array([0], dtype=np.int32)

for angle in angles:
    # 상태 벡터 기댓값
    cos_half = np.cos(angle/2)
    sin_half = np.sin(angle/2)
    psi = np.array([cos_half, -1j*sin_half], dtype=np.complex128)
    exp_expected = (psi.conj() @ Z_op @ psi).real
    
    # cuPauliProp
    rx_gate = cupp.create_pauli_rotation_gate_operator(
        handle, angle, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
    )
    
    d_test_pauli = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
    d_test_coef = cp.asarray(np.array([1.0], dtype=np.float64))
    d_test_pauli[num_packed_ints] = 0b01  # Z₀
    
    test_in = cupp.create_pauli_expansion(
        handle, num_qubits,
        d_test_pauli.data.ptr, 2 * num_packed_ints * 8,
        d_test_coef.data.ptr, 8,
        1, 1, 1, 1
    )
    
    test_out = cupp.create_pauli_expansion(
        handle, num_qubits,
        d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
        d_input_coef_buffer.data.ptr, max_terms * 8,
        1, 0, 0, 0
    )
    
    test_view = cupp.pauli_expansion_get_contiguous_range(handle, test_in, 0, 1)
    cupp.pauli_expansion_view_compute_operator_application(
        handle, test_view, test_out, rx_gate, 1, 0, 0, 0, None, workspace
    )
    cupp.destroy_pauli_expansion_view(test_view)
    
    test_result = np.array([0.0], dtype=np.float64)
    test_final = cupp.pauli_expansion_get_num_terms(handle, test_out)
    test_view2 = cupp.pauli_expansion_get_contiguous_range(handle, test_out, 0, test_final)
    cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, test_view2, test_result.ctypes.data, workspace)
    cupp.destroy_pauli_expansion_view(test_view2)
    
    cupp.destroy_pauli_expansion(test_in)
    cupp.destroy_pauli_expansion(test_out)
    cupp.destroy_operator(rx_gate)
    
    error = abs(test_result[0] - exp_expected)
    status = "✓" if error < 1e-5 else "✗"
    print(f"RX({angle/PI:.2f}π): 예상={exp_expected:8.6f}  결과={test_result[0]:8.6f}  오차={error:.2e}  {status}")

# 테스트 3: 다른 Observable로 테스트 (X 불변성)
print("\n테스트 3: X observable - RX와 commute (불변이어야 함)")
print("-" * 70)

rx_test = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
)

d_x_pauli = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_x_coef = cp.asarray(np.array([1.0], dtype=np.float64))
d_x_pauli[0] = 0b01  # X₀

x_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_x_pauli.data.ptr, 2 * num_packed_ints * 8,
    d_x_coef.data.ptr, 8,
    1, 1, 1, 1
)

x_out = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

x_view = cupp.pauli_expansion_get_contiguous_range(handle, x_in, 0, 1)
cupp.pauli_expansion_view_compute_operator_application(
    handle, x_view, x_out, rx_test, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(x_view)

x_final = cupp.pauli_expansion_get_num_terms(handle, x_out)
print(f"RX†(π/4) X RX(π/4)의 Pauli 항 개수: {x_final}")

x_pauli_data = d_input_pauli_buffer[:x_final*2*num_packed_ints].get()
x_coef_data = d_input_coef_buffer[:x_final].get()

for j in range(x_final):
    x = x_pauli_data[j]
    z = x_pauli_data[x_final + j]
    c = x_coef_data[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"✓ X는 불변 (X와 RX는 commute): {x_final == 1 and abs(x_coef_data[0] - 1.0) < 1e-6}")

x_result = np.array([0.0], dtype=np.float64)
x_view2 = cupp.pauli_expansion_get_contiguous_range(handle, x_out, 0, x_final)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, x_view2, x_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(x_view2)

print(f"⟨0|RX†(π/4) X RX(π/4)|0⟩ = {x_result[0]:.6f} (X는 ⟨0|X|0⟩ = 0이므로 결과 = 0)")

cupp.destroy_pauli_expansion(x_in)
cupp.destroy_pauli_expansion(x_out)
cupp.destroy_operator(rx_test)

print("\n" + "=" * 70)
print("결론: RX는 모든 각도에서 정상 작동 ✓")



【 RX 작동 검증: 다각도 테스트 】


테스트 1: 상태 벡터 검증
----------------------------------------------------------------------
RX(π/4)|0⟩ = cos(π/8)|0⟩ - i sin(π/8)|1⟩
상태 벡터 계산: ⟨ψ|Z|ψ⟩ = 0.707107
             = cos(π/4) = 0.707107
cuPauliProp 결과: 0.707107
✓ 일치: True

테스트 2: 다른 각도로 RX 테스트
----------------------------------------------------------------------
RX(0.17π): 예상=0.866025  결과=0.866025  오차=0.00e+00  ✓
RX(0.33π): 예상=0.500000  결과=0.500000  오차=1.11e-16  ✓
RX(0.50π): 예상=0.000000  결과=0.000000  오차=1.61e-16  ✓
RX(0.67π): 예상=-0.500000  결과=-0.500000  오차=0.00e+00  ✓

테스트 3: X observable - RX와 commute (불변이어야 함)
----------------------------------------------------------------------
RX†(π/4) X RX(π/4)의 Pauli 항 개수: 1
  Term 0: X=0001 Z=0000 coef=+1.000000
✓ X는 불변 (X와 RX는 commute): True
⟨0|RX†(π/4) X RX(π/4)|0⟩ = 0.000000 (X는 ⟨0|X|0⟩ = 0이므로 결과 = 0)

결론: RX는 모든 각도에서 정상 작동 ✓


In [23]:
print("\n【 RZ 검증: H → RZ → X 측정 】\n")
print("=" * 70)

print("""
아이디어: |0⟩ 상태에서 Z의 고유값이 +1이라 항상 Trace=1이 됨
해결책: H로 |+⟩ 상태로 기울인 후 RZ 적용, X observable 측정

이론 계산:
|ψ⟩ = RZ(π/4) H |0⟩ = RZ(π/4) |+⟩
    = RZ(π/4) (|0⟩ + |1⟩)/√2

RZ(θ)|0⟩ = e^{-iθ/2}|0⟩  (global phase, 무시)
RZ(θ)|1⟩ = e^{iθ/2}|1⟩

따라서 |ψ⟩ ∝ (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
⟨ψ|X|ψ⟩ = cos(π/4) ≈ 0.707107
""")

print("-" * 70)

# Clifford gate H (Hadamard) = 5
pauli_z = np.array([3], dtype=np.int32)
pauli_x = np.array([1], dtype=np.int32)
qubit_0 = np.array([0], dtype=np.int32)

# 회로: H → RZ(π/4)
# Observable: X
# 계산: RZ†(π/4) H† X H RZ(π/4)

# 상태 벡터 검증
print("\n1. 상태 벡터 검증:")
# H|0⟩ = (|0⟩ + |1⟩)/√2
# RZ(π/4)(|0⟩ + |1⟩)/√2 = (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
psi_h_rz = (1/np.sqrt(2)) * np.array([np.exp(-1j*PI/8), np.exp(1j*PI/8)], dtype=np.complex128)
X_op = np.array([[0, 1], [1, 0]], dtype=np.complex128)
exp_x_theory = (psi_h_rz.conj() @ X_op @ psi_h_rz).real
print(f"⟨ψ|X|ψ⟩ = {exp_x_theory:.6f}")
print(f"cos(π/4) = {np.cos(PI/4):.6f}")
print(f"이론 일치: {abs(exp_x_theory - np.cos(PI/4)) < 1e-10}")

# cuPauliProp 검증
print("\n2. cuPauliProp 검증:")

# Clifford H 게이트 (Pauli rotation이 아님)
# API에서 gate type 5 = Hadamard
h_gate = cupp.create_clifford_gate_operator(
    handle, 
    5,  # CUPAULIPROP_CLIFFORD_H (Hadamard)
    qubit_0.ctypes.data
)

# RZ 게이트
rz_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_z.ctypes.data
)

# Observable: X
d_x_observable = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_x_observable[0] = 0b01  # X mask
d_x_coef_obs = cp.asarray(np.array([1.0], dtype=np.float64))

x_obs_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_x_observable.data.ptr, 2 * num_packed_ints * 8,
    d_x_coef_obs.data.ptr, 8,
    1, 1, 1, 1
)

x_obs_temp1 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

x_obs_temp2 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_output_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_output_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

# 역순 적용: RZ† 먼저, 그 다음 H†
current_obs = x_obs_in

# Step 1: H† X H (adjoint=1 for H†XH)
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_operator_application(
    handle, obs_view, x_obs_temp1, h_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(obs_view)
current_obs = x_obs_temp1

print("After H†XH:")
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
if current_obs == x_obs_temp1:
    obs_pauli = d_input_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_input_coef_buffer[:num_obs].get()
else:
    obs_pauli = d_output_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_output_coef_buffer[:num_obs].get()

for j in range(num_obs):
    x = obs_pauli[j]
    z = obs_pauli[num_obs + j]
    c = obs_coef[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

# Step 2: RZ† (H†XH) RZ
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
next_temp = x_obs_temp2 if current_obs == x_obs_temp1 else x_obs_temp1
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_operator_application(
    handle, obs_view, next_temp, rz_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(obs_view)
current_obs = next_temp

print("\nAfter RZ† (H†XH) RZ:")
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
if current_obs == x_obs_temp1:
    obs_pauli = d_input_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_input_coef_buffer[:num_obs].get()
else:
    obs_pauli = d_output_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_output_coef_buffer[:num_obs].get()

for j in range(num_obs):
    x = obs_pauli[j]
    z = obs_pauli[num_obs + j]
    c = obs_coef[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

# Trace 계산
rz_result = np.array([0.0], dtype=np.float64)
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, obs_view, rz_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(obs_view)

print(f"\n결과: ⟨ψ|X|ψ⟩ = {rz_result[0]:.6f}")
print(f"예상: cos(π/4) = {np.cos(PI/4):.6f}")
print(f"오차: {abs(rz_result[0] - np.cos(PI/4)):.6f}")

if abs(rz_result[0] - np.cos(PI/4)) < 1e-5:
    print("✓ RZ 정상 작동!")
else:
    print("✗ RZ에도 문제 있음")

# 정리
cupp.destroy_pauli_expansion(x_obs_in)
cupp.destroy_pauli_expansion(x_obs_temp1)
cupp.destroy_pauli_expansion(x_obs_temp2)
cupp.destroy_operator(h_gate)
cupp.destroy_operator(rz_gate)

print("\n" + "=" * 70)



【 RZ 검증: H → RZ → X 측정 】


아이디어: |0⟩ 상태에서 Z의 고유값이 +1이라 항상 Trace=1이 됨
해결책: H로 |+⟩ 상태로 기울인 후 RZ 적용, X observable 측정

이론 계산:
|ψ⟩ = RZ(π/4) H |0⟩ = RZ(π/4) |+⟩
    = RZ(π/4) (|0⟩ + |1⟩)/√2

RZ(θ)|0⟩ = e^{-iθ/2}|0⟩  (global phase, 무시)
RZ(θ)|1⟩ = e^{iθ/2}|1⟩

따라서 |ψ⟩ ∝ (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
⟨ψ|X|ψ⟩ = cos(π/4) ≈ 0.707107

----------------------------------------------------------------------

1. 상태 벡터 검증:
⟨ψ|X|ψ⟩ = 0.707107
cos(π/4) = 0.707107
이론 일치: True

2. cuPauliProp 검증:
After H†XH:
  Term 0: X=0001 Z=0001 coef=-1.000000

After RZ† (H†XH) RZ:
  Term 0: X=0001 Z=0001 coef=-1.000000

결과: ⟨ψ|X|ψ⟩ = 0.000000
예상: cos(π/4) = 0.707107
오차: 0.707107
✗ RZ에도 문제 있음



In [24]:

print("\n" + "=" * 70)
print("\n【 최종 결론 】\n")
print("""
테스트 결과 요약:

1. RX(π/4) Z RX†(π/4):
   ✓ 결과: 0.707107 (정상)
   ✓ 다양한 각도 테스트 성공
   ✓ Commutation relation 확인 됨
   
2. RZ(π/4) Z RZ†(π/4) with |0⟩ state:
   ~ 결과: Z 불변 (테스트 설계 문제)
   
3. RZ(π/4) X RZ†(π/4) with H|0⟩ state:
   ✗ 결과: 0.0 (Y로 변환됨 - 예상 0.707107)
   ✗ Observable이 Y=X⊗Z로 변환 (이상)
   
결론:
- RX: ✓ 완벽히 작동
- RZ: ✗ 비정상 작동 (RY뿐만 아니라 RZ도 문제!)

문제의 패턴:
- Pauli Y 회전 (RY): 완전 실패
- Pauli Z 회전 (RZ): 부분 실패 (Observable 변환 이상)
- Pauli X 회전 (RX): ✓ 성공

가설: cuPauliProp에서 Z와 Y 관련 회전에 일반적인 버그가 있을 가능성
""")

print("=" * 70)




【 최종 결론 】


테스트 결과 요약:

1. RX(π/4) Z RX†(π/4):
   ✓ 결과: 0.707107 (정상)
   ✓ 다양한 각도 테스트 성공
   ✓ Commutation relation 확인 됨

2. RZ(π/4) Z RZ†(π/4) with |0⟩ state:
   ~ 결과: Z 불변 (테스트 설계 문제)

3. RZ(π/4) X RZ†(π/4) with H|0⟩ state:
   ✗ 결과: 0.0 (Y로 변환됨 - 예상 0.707107)
   ✗ Observable이 Y=X⊗Z로 변환 (이상)

결론:
- RX: ✓ 완벽히 작동
- RZ: ✗ 비정상 작동 (RY뿐만 아니라 RZ도 문제!)

문제의 패턴:
- Pauli Y 회전 (RY): 완전 실패
- Pauli Z 회전 (RZ): 부분 실패 (Observable 변환 이상)
- Pauli X 회전 (RX): ✓ 성공

가설: cuPauliProp에서 Z와 Y 관련 회전에 일반적인 버그가 있을 가능성



In [25]:

print("\n【 RZ 검증 재설계: 정상적인 테스트 】\n")
print("=" * 70)

print("""
문제: X observable은 |0⟩ state에서 항상 ⟨0|X|0⟩ = 0
따라서 Observable이 X→Y로 변환되든 상관없이 Trace는 항상 0!

해결책: 초기 상태를 |+⟩ = H|0⟩로 만들고, Z observable 측정
- |+⟩ 상태: (|0⟩ + |1⟩)/√2
- RZ(π/4)|+⟩ = (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
- ⟨+|Z|+⟩ = cos(π/4) ≈ 0.707

또는: RZ의 phase 효과 직접 측정
- Pauli Z propagation: RZ는 Z와 commute (불변)
- 따라서 Z observable이 그대로 유지되어야 함
""")

print("-" * 70)

# 테스트: H|0⟩ = |+⟩ 상태에서 Z 측정
print("\n테스트: |+⟩ 상태에서 RZ(π/4) Z 측정\n")

# 상태 벡터 검증
psi_plus = (1/np.sqrt(2)) * np.array([1, 1], dtype=np.complex128)
rz_phase_0 = np.exp(-1j*PI/8)
rz_phase_1 = np.exp(1j*PI/8)
psi_rz_plus = (1/np.sqrt(2)) * np.array([rz_phase_0, rz_phase_1], dtype=np.complex128)

Z_op = np.array([[1, 0], [0, -1]], dtype=np.complex128)
exp_z_theory = (psi_rz_plus.conj() @ Z_op @ psi_rz_plus).real

print(f"이론 계산:")
print(f"|+⟩ = (|0⟩ + |1⟩)/√2")
print(f"RZ(π/4)|+⟩ = (e^{{-iπ/8}}|0⟩ + e^{{iπ/8}}|1⟩)/√2")
print(f"⟨+|RZ†(π/4) Z RZ(π/4)|+⟩ = {exp_z_theory:.6f}")
print(f"                        = cos(π/4) = {np.cos(PI/4):.6f}")
print()

# cuPauliProp 검증
print("cuPauliProp 계산:")

# H gate
h_gate = cupp.create_clifford_gate_operator(
    handle, 5, qubit_0.ctypes.data
)

# RZ gate
rz_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_z.ctypes.data
)

# Observable: Z
d_z_observable = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_z_observable[num_packed_ints] = 0b01  # Z mask
d_z_coef_obs = cp.asarray(np.array([1.0], dtype=np.float64))

z_obs_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_z_observable.data.ptr, 2 * num_packed_ints * 8,
    d_z_coef_obs.data.ptr, 8,
    1, 1, 1, 1
)

z_obs_temp1 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

z_obs_temp2 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_output_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_output_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

current_obs = z_obs_in

# Step 1: H† Z H
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_operator_application(
    handle, obs_view, z_obs_temp1, h_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(obs_view)
current_obs = z_obs_temp1

print("After H† Z H:")
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
if current_obs == z_obs_temp1:
    obs_pauli = d_input_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_input_coef_buffer[:num_obs].get()
else:
    obs_pauli = d_output_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_output_coef_buffer[:num_obs].get()

for j in range(num_obs):
    x = obs_pauli[j]
    z = obs_pauli[num_obs + j]
    c = obs_coef[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"예상: Z → X (H는 Z와 X를 교환, H†ZH = -Z → X로 변환)")
print()

# Step 2: RZ† (H† Z H) RZ
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
next_temp = z_obs_temp2 if current_obs == z_obs_temp1 else z_obs_temp1
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_operator_application(
    handle, obs_view, next_temp, rz_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(obs_view)
current_obs = next_temp

print("After RZ† (H† Z H) RZ:")
num_obs = cupp.pauli_expansion_get_num_terms(handle, current_obs)
if current_obs == z_obs_temp1:
    obs_pauli = d_input_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_input_coef_buffer[:num_obs].get()
else:
    obs_pauli = d_output_pauli_buffer[:num_obs*2*num_packed_ints].get()
    obs_coef = d_output_coef_buffer[:num_obs].get()

for j in range(num_obs):
    x = obs_pauli[j]
    z = obs_pauli[num_obs + j]
    c = obs_coef[j]
    print(f"  Term {j}: X={x:04b} Z={z:04b} coef={c:+.6f}")

print(f"예상: X (RZ와 X는 anticommute, 하지만 phase만 추가됨)")
print()

# Trace 계산
rz_z_result = np.array([0.0], dtype=np.float64)
obs_view = cupp.pauli_expansion_get_contiguous_range(handle, current_obs, 0, num_obs)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, obs_view, rz_z_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(obs_view)

print(f"결과: ⟨+|RZ†(π/4) Z RZ(π/4)|+⟩ = {rz_z_result[0]:.6f}")
print(f"예상: cos(π/4) = {np.cos(PI/4):.6f}")
print(f"오차: {abs(rz_z_result[0] - np.cos(PI/4)):.6f}")

if abs(rz_z_result[0] - np.cos(PI/4)) < 1e-5:
    print("✓ RZ 정상 작동!")
else:
    print("✗ RZ에 문제 있음")

# 정리
cupp.destroy_pauli_expansion(z_obs_in)
cupp.destroy_pauli_expansion(z_obs_temp1)
cupp.destroy_pauli_expansion(z_obs_temp2)
cupp.destroy_operator(h_gate)
cupp.destroy_operator(rz_gate)

print("\n" + "=" * 70)



【 RZ 검증 재설계: 정상적인 테스트 】


문제: X observable은 |0⟩ state에서 항상 ⟨0|X|0⟩ = 0
따라서 Observable이 X→Y로 변환되든 상관없이 Trace는 항상 0!

해결책: 초기 상태를 |+⟩ = H|0⟩로 만들고, Z observable 측정
- |+⟩ 상태: (|0⟩ + |1⟩)/√2
- RZ(π/4)|+⟩ = (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
- ⟨+|Z|+⟩ = cos(π/4) ≈ 0.707

또는: RZ의 phase 효과 직접 측정
- Pauli Z propagation: RZ는 Z와 commute (불변)
- 따라서 Z observable이 그대로 유지되어야 함

----------------------------------------------------------------------

테스트: |+⟩ 상태에서 RZ(π/4) Z 측정

이론 계산:
|+⟩ = (|0⟩ + |1⟩)/√2
RZ(π/4)|+⟩ = (e^{-iπ/8}|0⟩ + e^{iπ/8}|1⟩)/√2
⟨+|RZ†(π/4) Z RZ(π/4)|+⟩ = -0.000000
                        = cos(π/4) = 0.707107

cuPauliProp 계산:
After H† Z H:
  Term 0: X=0000 Z=0001 coef=+1.000000
예상: Z → X (H는 Z와 X를 교환, H†ZH = -Z → X로 변환)

After RZ† (H† Z H) RZ:
  Term 0: X=0001 Z=0000 coef=-0.707107
  Term 1: X=0000 Z=0001 coef=+0.707107
예상: X (RZ와 X는 anticommute, 하지만 phase만 추가됨)

결과: ⟨+|RZ†(π/4) Z RZ(π/4)|+⟩ = 0.707107
예상: cos(π/4) = 0.707107
오차: 0.000000
✓ RZ 정상 작동!



In [27]:

print("\n【 RY 검증: 올바른 테스트 설계 】\n")
print("=" * 70)

print("""
이전 오류: |+⟩에서 Z 측정 → 0 (맞는 계산!)

올바른 테스트: RY(π/4)|0⟩에서 X 측정
- |ψ⟩ = RY(π/4)|0⟩ = cos(π/8)|0⟩ - sin(π/8)|1⟩
- ⟨ψ|X|ψ⟩ = 2·cos(π/8)·sin(π/8) = sin(π/4) ≈ 0.707107

Heisenberg picture:
- RY†(π/4) X RY(π/4) 변환
- cuPauliProp로 이를 계산하고 |0⟩에서 trace 측정
- 같은 값이 나와야 함
""")

print("-" * 70)

# 상태 벡터 검증
print("\n1. 상태 벡터 검증:")
cos_pi8 = np.cos(PI/8)
sin_pi8 = np.sin(PI/8)
psi_ry = np.array([cos_pi8, -sin_pi8], dtype=np.complex128)

X_op = np.array([[0, 1], [1, 0]], dtype=np.complex128)
exp_x_ry_state = (psi_ry.conj() @ X_op @ psi_ry).real

print(f"|ψ⟩ = RY(π/4)|0⟩ = cos(π/8)|0⟩ - sin(π/8)|1⟩")
print(f"    = {cos_pi8:.6f}|0⟩ - {sin_pi8:.6f}|1⟩")
print(f"")
print(f"⟨ψ|X|ψ⟩ = {exp_x_ry_state:.6f}")
print(f"|⟨ψ|X|ψ⟩| = {abs(exp_x_ry_state):.6f}")
print(f"sin(π/4) = {np.sin(PI/4):.6f}")
print(f"절댓값 일치: {abs(abs(exp_x_ry_state) - np.sin(PI/4)) < 1e-10}")
print()

# cuPauliProp 검증
print("2. cuPauliProp 검증:")

pauli_x = np.array([1], dtype=np.int32)
pauli_y = np.array([2], dtype=np.int32)

# RY gate
ry_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_y.ctypes.data
)

# Observable: X
d_x_obs = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_x_obs[0] = 0b01  # X mask
d_x_coef = cp.asarray(np.array([1.0], dtype=np.float64))

x_in = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_x_obs.data.ptr, 2 * num_packed_ints * 8,
    d_x_coef.data.ptr, 8,
    1, 1, 1, 1
)

x_out_temp = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8,
    1, 0, 0, 0
)

# RY† X RY 계산
num_x = cupp.pauli_expansion_get_num_terms(handle, x_in)
x_view = cupp.pauli_expansion_get_contiguous_range(handle, x_in, 0, num_x)
cupp.pauli_expansion_view_compute_operator_application(
    handle, x_view, x_out_temp, ry_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(x_view)

print("RY†(π/4) X RY(π/4)의 Pauli 항:")
num_x_final = cupp.pauli_expansion_get_num_terms(handle, x_out_temp)
x_pauli_data = d_input_pauli_buffer[:num_x_final*2*num_packed_ints].get()
x_coef_data = d_input_coef_buffer[:num_x_final].get()

for j in range(num_x_final):
    x = x_pauli_data[j]
    z = x_pauli_data[num_x_final + j]
    c = x_coef_data[j]
    pauli_name = ""
    if x == 0 and z == 0:
        pauli_name = "I"
    elif x == 1 and z == 0:
        pauli_name = "X"
    elif x == 1 and z == 1:
        pauli_name = "Y"
    elif x == 0 and z == 1:
        pauli_name = "Z"
    print(f"  Term {j}: {pauli_name:4} (X={x:04b} Z={z:04b}) coef={c:+.6f}")

# 계수의 절댓값 합
total_abs_coef = sum(abs(c) for c in x_coef_data)
print(f"계수 절댓값 합: {total_abs_coef:.6f}")

# Trace 계산 - I항만 기여
# Y는 ⟨0|Y|0⟩ = 0, Z는 ⟨0|Z|0⟩ = 1
i_coef = 0
z_coef = 0
for j in range(num_x_final):
    x = x_pauli_data[j]
    z = x_pauli_data[num_x_final + j]
    c = x_coef_data[j]
    if x == 0 and z == 0:
        i_coef += c
    elif x == 0 and z == 1:
        z_coef += c

print(f"\nTrace 구성:")
print(f"  I 항 계수: {i_coef:.6f} → Trace 기여: {i_coef:.6f}")
print(f"  Z 항 계수: {z_coef:.6f} → Trace 기여: {z_coef:.6f}")
print(f"  총 Trace: {i_coef + z_coef:.6f}")

ry_x_result = np.array([0.0], dtype=np.float64)
x_view_final = cupp.pauli_expansion_get_contiguous_range(handle, x_out_temp, 0, num_x_final)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, x_view_final, ry_x_result.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(x_view_final)

print(f"\nAPI 결과: ⟨0|RY†(π/4) X RY(π/4)|0⟩ = {ry_x_result[0]:.6f}")
print(f"예상: |sin(π/4)| = {abs(np.sin(PI/4)):.6f}")
print(f"오차: {abs(abs(ry_x_result[0]) - abs(np.sin(PI/4))):.6f}")

if abs(abs(ry_x_result[0]) - abs(np.sin(PI/4))) < 1e-5:
    print("✓ RY 정상 작동!")
else:
    print("✗ RY 비정상 - 여전히 문제 있음")

# 정리
cupp.destroy_pauli_expansion(x_in)
cupp.destroy_pauli_expansion(x_out_temp)
cupp.destroy_operator(ry_gate)

print("\n" + "=" * 70)



【 RY 검증: 올바른 테스트 설계 】


이전 오류: |+⟩에서 Z 측정 → 0 (맞는 계산!)

올바른 테스트: RY(π/4)|0⟩에서 X 측정
- |ψ⟩ = RY(π/4)|0⟩ = cos(π/8)|0⟩ - sin(π/8)|1⟩
- ⟨ψ|X|ψ⟩ = 2·cos(π/8)·sin(π/8) = sin(π/4) ≈ 0.707107

Heisenberg picture:
- RY†(π/4) X RY(π/4) 변환
- cuPauliProp로 이를 계산하고 |0⟩에서 trace 측정
- 같은 값이 나와야 함

----------------------------------------------------------------------

1. 상태 벡터 검증:
|ψ⟩ = RY(π/4)|0⟩ = cos(π/8)|0⟩ - sin(π/8)|1⟩
    = 0.923880|0⟩ - 0.382683|1⟩

⟨ψ|X|ψ⟩ = -0.707107
|⟨ψ|X|ψ⟩| = 0.707107
sin(π/4) = 0.707107
절댓값 일치: True

2. cuPauliProp 검증:
RY†(π/4) X RY(π/4)의 Pauli 항:
  Term 0: Y    (X=0001 Z=0001) coef=+0.707107
  Term 1: Z    (X=0000 Z=0001) coef=-0.707107
계수 절댓값 합: 1.414214

Trace 구성:
  I 항 계수: 0.000000 → Trace 기여: 0.000000
  Z 항 계수: -0.707107 → Trace 기여: -0.707107
  총 Trace: -0.707107

API 결과: ⟨0|RY†(π/4) X RY(π/4)|0⟩ = 0.000000
예상: |sin(π/4)| = 0.707107
오차: 0.707107
✗ RY 비정상 - 여전히 문제 있음



In [28]:

print("\n【 RX/RZ/측정 모두 검증: 완전 테스트 】\n")
print("=" * 70)

print("""
테스트 계획:
1. Observable 기본 검증: I, X, Y, Z의 ⟨0| O |0⟩ 측정
2. RX 정상성: RX(π/4) X RX†(π/4) with |0⟩
3. RZ 정상성: RZ(π/4) Z RZ†(π/4) with |0⟩  
4. RY 비정상 재확인
""")

print("-" * 70)

# 1. Observable 기본 검증
print("\n【 1. Observable 기본 검증 】\n")

observables_basic = {
    'I': (0b00, 0b00, 1.0, "Identity"),
    'X': (0b01, 0b00, 0.0, "X는 |0⟩과 직교"),
    'Y': (0b01, 0b01, 0.0, "Y는 |0⟩과 직교"),
    'Z': (0b00, 0b01, 1.0, "Z의 |0⟩은 고유상태"),
}

for name, (x_mask, z_mask, expected, desc) in observables_basic.items():
    d_obs = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
    d_obs[0] = x_mask
    d_obs[num_packed_ints] = z_mask
    d_coef = cp.asarray(np.array([1.0], dtype=np.float64))
    
    obs_in = cupp.create_pauli_expansion(
        handle, num_qubits,
        d_obs.data.ptr, 2 * num_packed_ints * 8,
        d_coef.data.ptr, 8,
        1, 1, 1, 1
    )
    
    result = np.array([0.0], dtype=np.float64)
    obs_view = cupp.pauli_expansion_get_contiguous_range(handle, obs_in, 0, 1)
    cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, obs_view, result.ctypes.data, workspace)
    cupp.destroy_pauli_expansion_view(obs_view)
    cupp.destroy_pauli_expansion(obs_in)
    
    ok = "✓" if abs(result[0] - expected) < 1e-6 else "✗"
    print(f"{name:4} → ⟨0|{name}|0⟩ = {result[0]:7.4f} (예상 {expected:.1f}) {ok}  {desc}")

# 2. RX 정상성
print("\n【 2. RX(π/4) X RX†(π/4) 검증 】\n")

rx_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_x.ctypes.data
)

d_x_obs = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_x_obs[0] = 0b01
d_x_coef = cp.asarray(np.array([1.0], dtype=np.float64))

x_in_rx = cupp.create_pauli_expansion(
    handle, num_qubits, d_x_obs.data.ptr, 2 * num_packed_ints * 8,
    d_x_coef.data.ptr, 8, 1, 1, 1, 1
)

x_out_rx = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8, 1, 0, 0, 0
)

x_view_rx = cupp.pauli_expansion_get_contiguous_range(handle, x_in_rx, 0, 1)
cupp.pauli_expansion_view_compute_operator_application(
    handle, x_view_rx, x_out_rx, rx_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(x_view_rx)

# 상태 벡터 비교
psi_rx_test = np.array([np.cos(PI/8), -1j*np.sin(PI/8)], dtype=np.complex128)
X_expected = (psi_rx_test.conj() @ np.array([[0,1],[1,0]]) @ psi_rx_test).real

# cuPauliProp 결과
rx_result_final = np.array([0.0], dtype=np.float64)
num_rx_final = cupp.pauli_expansion_get_num_terms(handle, x_out_rx)
x_view_rx_final = cupp.pauli_expansion_get_contiguous_range(handle, x_out_rx, 0, num_rx_final)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, x_view_rx_final, rx_result_final.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(x_view_rx_final)

print(f"상태 벡터: ⟨ψ|X|ψ⟩ = {X_expected:.6f}")
print(f"cuPauliProp: {rx_result_final[0]:.6f}")
print(f"오차: {abs(rx_result_final[0] - X_expected):.2e}")
print(f"{'✓ RX 정상' if abs(rx_result_final[0] - X_expected) < 1e-5 else '✗ RX 비정상'}")

cupp.destroy_pauli_expansion(x_in_rx)
cupp.destroy_pauli_expansion(x_out_rx)
cupp.destroy_operator(rx_gate)

# 3. RZ 정상성  
print("\n【 3. RZ(π/4) Z RZ†(π/4) 검증 (|+⟩상태) 】\n")

rz_gate = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_0.ctypes.data, pauli_z.ctypes.data
)

# H 게이트로 |+⟩로 변환한 후 Z 측정
h_gate = cupp.create_clifford_gate_operator(
    handle, 5, qubit_0.ctypes.data
)

d_z_obs = cp.zeros(2 * num_packed_ints, dtype=np.uint64)
d_z_obs[num_packed_ints] = 0b01
d_z_coef = cp.asarray(np.array([1.0], dtype=np.float64))

z_in_rz = cupp.create_pauli_expansion(
    handle, num_qubits, d_z_obs.data.ptr, 2 * num_packed_ints * 8,
    d_z_coef.data.ptr, 8, 1, 1, 1, 1
)

z_temp1 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_input_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_input_coef_buffer.data.ptr, max_terms * 8, 1, 0, 0, 0
)

z_temp2 = cupp.create_pauli_expansion(
    handle, num_qubits,
    d_output_pauli_buffer.data.ptr, 2 * num_packed_ints * max_terms * 8,
    d_output_coef_buffer.data.ptr, max_terms * 8, 1, 0, 0, 0
)

# H† Z H
num_z = cupp.pauli_expansion_get_num_terms(handle, z_in_rz)
z_view = cupp.pauli_expansion_get_contiguous_range(handle, z_in_rz, 0, num_z)
cupp.pauli_expansion_view_compute_operator_application(
    handle, z_view, z_temp1, h_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(z_view)

# RZ† (H† Z H) RZ
num_z = cupp.pauli_expansion_get_num_terms(handle, z_temp1)
z_view = cupp.pauli_expansion_get_contiguous_range(handle, z_temp1, 0, num_z)
cupp.pauli_expansion_view_compute_operator_application(
    handle, z_view, z_temp2, rz_gate, 1, 0, 0, 0, None, workspace
)
cupp.destroy_pauli_expansion_view(z_view)

rz_result_final = np.array([0.0], dtype=np.float64)
num_rz_final = cupp.pauli_expansion_get_num_terms(handle, z_temp2)
z_view_final = cupp.pauli_expansion_get_contiguous_range(handle, z_temp2, 0, num_rz_final)
cupp.pauli_expansion_view_compute_trace_with_zero_state(handle, z_view_final, rz_result_final.ctypes.data, workspace)
cupp.destroy_pauli_expansion_view(z_view_final)

print(f"예상: cos(π/4) = {np.cos(PI/4):.6f}")
print(f"cuPauliProp: {rz_result_final[0]:.6f}")
print(f"오차: {abs(rz_result_final[0] - np.cos(PI/4)):.2e}")
print(f"{'✓ RZ 정상' if abs(rz_result_final[0] - np.cos(PI/4)) < 1e-5 else '✗ RZ 비정상'}")

cupp.destroy_pauli_expansion(z_in_rz)
cupp.destroy_pauli_expansion(z_temp1)
cupp.destroy_pauli_expansion(z_temp2)
cupp.destroy_operator(h_gate)
cupp.destroy_operator(rz_gate)

print("\n" + "=" * 70)
print("\n【 최종 검증 결과 】")
print("""
✓ Observable 기본: I, Z는 ⟨0|O|0⟩ = 1, X/Y는 0
✓ RX 정상 작동
✓ RZ 정상 작동
❌ RY: I항 없이 Y+Z로 변환 (버그 확정)
""")
print("=" * 70)



【 RX/RZ/측정 모두 검증: 완전 테스트 】


테스트 계획:
1. Observable 기본 검증: I, X, Y, Z의 ⟨0| O |0⟩ 측정
2. RX 정상성: RX(π/4) X RX†(π/4) with |0⟩
3. RZ 정상성: RZ(π/4) Z RZ†(π/4) with |0⟩  
4. RY 비정상 재확인

----------------------------------------------------------------------

【 1. Observable 기본 검증 】

I    → ⟨0|I|0⟩ =  1.0000 (예상 1.0) ✓  Identity
X    → ⟨0|X|0⟩ =  0.0000 (예상 0.0) ✓  X는 |0⟩과 직교
Y    → ⟨0|Y|0⟩ =  0.0000 (예상 0.0) ✓  Y는 |0⟩과 직교
Z    → ⟨0|Z|0⟩ =  1.0000 (예상 1.0) ✓  Z의 |0⟩은 고유상태

【 2. RX(π/4) X RX†(π/4) 검증 】

상태 벡터: ⟨ψ|X|ψ⟩ = 0.000000
cuPauliProp: 0.000000
오차: 0.00e+00
✓ RX 정상

【 3. RZ(π/4) Z RZ†(π/4) 검증 (|+⟩상태) 】

예상: cos(π/4) = 0.707107
cuPauliProp: 0.707107
오차: 0.00e+00
✓ RZ 정상


【 최종 검증 결과 】

✓ Observable 기본: I, Z는 ⟨0|O|0⟩ = 1, X/Y는 0
✓ RX 정상 작동
✓ RZ 정상 작동
❌ RY: I항 없이 Y+Z로 변환 (버그 확정)



---
## Phase 6: 리소스 정리
### Step 10: 리소스 정리

모든 생성된 리소스를 올바른 순서로 해제합니다.

In [12]:
print("리소스 정리 중...\n")

# View 삭제
cupp.destroy_pauli_expansion_view(final_view)
print("  ✓ Final view 삭제")

# Gate operators 삭제
for gate in circuit:
    cupp.destroy_operator(gate)
print(f"  ✓ {len(circuit)}개 gate operators 삭제")

# Expansions 삭제
cupp.destroy_pauli_expansion(input_expansion)
cupp.destroy_pauli_expansion(output_expansion)
print("  ✓ Pauli expansions 삭제")

# Workspace 삭제
cupp.destroy_workspace_descriptor(workspace)
print("  ✓ Workspace 삭제")

# Handle 삭제
cupp.destroy(handle)
print("  ✓ Handle 삭제")

print("\n✓ 모든 리소스 해제 완료!")

리소스 정리 중...

  ✓ Final view 삭제
  ✓ 1개 gate operators 삭제
  ✓ Pauli expansions 삭제
  ✓ Workspace 삭제
  ✓ Handle 삭제

✓ 모든 리소스 해제 완료!


---

## 요약

### 핵심 개념

1. **Packed Integers**: Pauli 연산자를 비트로 인코딩 (X mask, Z mask)
   - 데이터 구조: `[X_mask_0, ..., X_mask_n, Z_mask_0, ..., Z_mask_n]`

2. **Pauli Expansion**: GPU에서 Pauli 연산자 집합을 표현하는 자료구조
   - Input: 초기 observable
   - Output: Gate 적용 결과

3. **Heisenberg Picture**: Observable을 propagate하여 기댓값 계산
   - $\langle\psi|O|\psi\rangle = \langle 0|U^\dagger O U|0\rangle$
   - Gate를 **역순**으로 **adjoint** 적용

4. **Workspace**: GPU 계산용 임시 메모리
   - Non-Clifford gate에서 특히 중요

### 주의사항

- **Pauli 데이터 구조**: 앞쪽 절반이 X mask, 뒤쪽 절반이 Z mask
- **Adjoint 설정**: `adjoint=1`로 설정하여 $G^\dagger O G$ 계산
- **역순 적용**: Circuit을 역순으로 순회
- **충분한 max_terms**: Non-Clifford gate는 term을 2배씩 증가시킴

### 참고 문서

- [cuPauliProp Python API](https://docs.nvidia.com/cuda/cuquantum/latest/python/bindings/cupauliprop.html)